In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-10 17:19:17--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-04-10 17:19:17 (6.55 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

In [37]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])

vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| REAKC26P07MDN|          5|            0|          0|   N|                Y|
|R3NU7OMZ4HQIEG|          2|            0|          1|   N|                Y|
|R14QJW3XF8QO1P|          5|            0|          0|   N|                Y|
|R2HB7AX0394ZGY|          5|            0|          0|   N|                Y|
| RGKMPDQGSAHR3|          5|            0|          0|   N|                Y|
|R1DJCVPQGCV66E|          5|            0|          0|   N|                Y|
|R3V52EAWLPBFQG|          3|            0|          0|   N|                Y|
|R3DKO8J1J28QBI|          2|            0|          0|   N|                Y|
| R764DBXGRNECG|          5|            1|          1|   N|                N|
| RW1853GAT0Z9F|          5|            0|          0|   N|     

In [38]:
reviews_df = vine_df.filter("total_votes>=20")

reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|
|R24QM6D7FEDZ5M|          2|           36|         43|   N|                Y|
|R3A1VQ3SQDXEJ3|          5|           20|         20|   N|                Y|
|  R39GSNW76GYF|          1|           20|         23|   N|                Y|
| RPJLR6MFDPXXE|          5|           35|         36|   N|     

In [39]:
#Number of helpful_votes divided by total_votes is equal to or greater than 50%
filtered_reviews_df = reviews_df.filter("(helpful_votes/total_votes)*100 >= 50")
filtered_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|
|R24QM6D7FEDZ5M|          2|           36|         43|   N|                Y|
|R3A1VQ3SQDXEJ3|          5|           20|         20|   N|                Y|
|  R39GSNW76GYF|          1|           20|         23|   N|                Y|
| RPJLR6MFDPXXE|          5|           35|         36|   N|     

In [40]:
#dataframe for paid
paid_df = filtered_reviews_df.filter("(vine) == 'Y'")
paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3A71VR1JZD8WF|          2|           27|         30|   Y|                N|
|R16OMUJIGI18JZ|          5|           72|         72|   Y|                N|
|R3TS8ZP2FHQ9XR|          5|           39|         42|   Y|                N|
|R2MHP919VZN7DI|          5|           29|         30|   Y|                N|
| RD2BCTVS59A5L|          2|           20|         20|   Y|                N|
|R1JUJLXQ2CMWKF|          4|           25|         26|   Y|                N|
|R2T7YE0IFI6N9L|          3|           24|         25|   Y|                N|
|R25FI3J7WWOYXM|          3|           55|         58|   Y|                N|
|R3P3U4EOWIUD50|          3|           27|         36|   Y|                N|
|R3C7MKX7MO6XUC|          5|          216|        219|   Y|     

In [41]:
#dataframe for unpaid
unpaid_df = filtered_reviews_df.filter("(vine) == 'N'")
unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|
|R24QM6D7FEDZ5M|          2|           36|         43|   N|                Y|
|R3A1VQ3SQDXEJ3|          5|           20|         20|   N|                Y|
|  R39GSNW76GYF|          1|           20|         23|   N|                Y|
| RPJLR6MFDPXXE|          5|           35|         36|   N|     

In [49]:
#total number of reviews
total_reviews = filtered_reviews_df.count()
print("There are",total_reviews,"total reviews.")

#total number of 5-star reviews
five_stars = filtered_reviews_df.filter("(star_rating) == 5")
five_star_reviews = five_stars.count()
print("There are",five_star_reviews,"total five-star reviews.")

percentage_five_stars = (five_star_reviews / total_reviews) * 100
print(percentage_five_stars,"% of total reviews are five-stars.")

#percentage of 5-star reviews for paid

five_stars_paid = five_stars.filter("(vine) == 'Y'").count() 
print("Out of the total five-star reviews, there are", five_stars_paid,"five-star reviews from Vine users.")

five_stars_paid_total = (five_stars_paid / five_star_reviews) * 100
print(five_stars_paid_total,"% of total five-star reviews are from paid Vine users.")

#percentage of 5-star reviews for unpaid
five_stars_unpaid = five_stars.filter("(vine) == 'N'").count()
print("Out of total five-star reviews, there are",five_stars_unpaid,"five-star reviews from non-Vine users.")

five_stars_unpaid_total = (five_stars_unpaid / five_star_reviews) * 100
print(five_stars_unpaid_total,"% of total five-star reviews come from unpaid users.")

There are 38010 total reviews.
There are 20677 total five-star reviews.
54.39884240989213 % of total reviews are five-stars.
Out of the total five-star reviews, there are 65 five-star reviews from Vine users.
0.31435894955747934 % of total five-star reviews are from paid Vine users.
Out of total five-star reviews, there are 20612 five-star reviews from non-Vine users.
99.68564105044251 % of total five-star reviews come from unpaid users.
